# CASM docker controlled through TINC

Start the CASM docker container with a command like:

```docker run -P --network casm_docker --hostname casm_ --rm -it -v c:\Users\Andres\source\repos:/shared casmcode/casm bash```

(You need to have created the docker bridge network 'casm_docker' running: ```casm network create casm_docker```) 

Find running casm docker container:

In [ ]:
from subprocess import check_output
docker_list = check_output(["docker", "ps", '--format', '{{.ID}}\t{{.Image}}\t{{.Names}}']).decode('ascii').splitlines()
for c in docker_list:
    if c.split('\t')[1]== 'casmcode/casm':
        container_name = c.split('\t')[2]
        
container_name

In [ ]:
from tinc import *

In [ ]:
tclient = TincClient()

Set up parameters

In [ ]:
settings_file_name = tclient.create_parameter(ParameterString, "settings_file_name", default_value="genetic_alg_settings.json")
hall_of_fame_index = tclient.create_parameter(ParameterInt,"hall_of_fame_index", default_value=0)
fit_dir = tclient.create_parameter(ParameterString,"fit_dir", default_value='')

Prepare first processor

In [ ]:
proc = ProcessorScript("check")
proc.command = "docker exec " + container_name + " casm-learn"
# Capture the output of the command to file
proc.capture_output()
# Register paramters with processor. Changes trigger computation
proc.register_parameter(hall_of_fame_index)
proc.register_parameter(settings_file_name)
# Define the command line argument template
# names within '%%' that match parameter names will be replaced by their value
proc.set_argument_template("-s %%settings_file_name%% --checkhull --indiv %%hall_of_fame_index%%")

# The output will be managed by a diskbuffer, to update data everywhere
db = DiskBufferText("check_buffer", "check.0", "out/", "/shared")
tclient.register_disk_buffer(db)
# You can set a disk buffer to be the output of a Processor
proc.output_files = [db]

# Because we want to change directory and output file on every run,
# We define a 'prepare' function
def prepare_check(p):
    p.running_dir = fit_dir.value
    db.set_base_filename(f"check.{hall_of_fame_index.value}")
    print(f"Set output to: {p.output_files[0]}")
    #print(p._get_arguments())
    return True
# This function will be called right before calling the command for ProcessorScript
proc.prepare = prepare_check
#proc.debug = True

In [ ]:
# Processor 2
proc2 = ProcessorScript("select")
proc2.command = "docker exec " + container_name + " casm-learn"
proc2.capture_output()
db2 = DiskBufferText("select_buffer","select_fit_eci.out", "out/", "/shared")
tclient.register_disk_buffer(db2)
proc2.set_argument_template("-s %%settings_file_name%% --select %%hall_of_fame_index%%")
proc2.register_parameter(hall_of_fame_index)
proc2.register_parameter(settings_file_name)

def prepare_select(p):
    p.running_dir = fit_dir.value
    return True

proc2.prepare = prepare_select
#proc2.debug = True

In [ ]:
# Processor 3
proc3 = ProcessorScript("fit_eci")
proc3.command = "docker exec " + container_name + " casm"
db3 = DiskBufferText("fit_eci_buffer","select_fit_eci.out", "out/", "/shared")
tclient.register_disk_buffer(db3)
proc3.output_files = [db]
proc3.set_argument_template("query -k comp formation_energy hull_dist clex clex_hulldist -o %%:OUTFILE:0%%")
proc3.register_parameter(hall_of_fame_index)
proc3.register_parameter(settings_file_name)

def prepare_fit(p):
    p.running_dir = fit_dir.value
    return True
proc3.prepare = prepare_fit
#proc3.debug = True

In [ ]:
from ipywidgets import VBox

v = VBox([settings_file_name.interactive_widget(), hall_of_fame_index.interactive_widget(), fit_dir.interactive_widget() ])
v